<a href="https://colab.research.google.com/github/rohhj622/windPowerPrediction/blob/master/code1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 데이터 불러오기

In [225]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import seaborn as sns
from datetime import datetime

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split
from sklearn import model_selection
from keras import models, layers, Sequential
from keras.layers import LSTM, Dropout, Dense, Activation, Input, SimpleRNN

## 1) 구글 드라이브 연결

In [226]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## 3) 데이터 불러오기 시작

In [227]:
allData = pd.read_csv(r'/content/gdrive/My Drive/Colab Notebooks/preprocessing_Data.csv') # 컬럼명 숫자로 들어가게
allData = allData.drop('Unnamed: 0',1)
allData

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem
0,2013-01-01 00:00:00,2013-01-01 00:00:00,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0
1,2013-01-01 01:00:00,2013-01-01 00:00:00,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0
2,2013-01-01 02:00:00,2013-01-01 00:00:00,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0
3,2013-01-01 03:00:00,2013-01-01 00:00:00,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3
4,2013-01-01 04:00:00,2013-01-01 00:00:00,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
52579,2018-12-31 19:00:00,2018-12-31 00:00:00,19:00,8699,4.3,4.7,340,58.0,4.8,-3.2,1032.1,1034.7,3.3
52580,2018-12-31 20:00:00,2018-12-31 00:00:00,20:00,10694,3.9,4.5,360,61.0,4.9,-2.9,1032.0,1034.6,2.9
52581,2018-12-31 21:00:00,2018-12-31 00:00:00,21:00,5494,3.5,4.0,360,59.0,4.6,-3.7,1031.9,1034.5,1.9
52582,2018-12-31 22:00:00,2018-12-31 00:00:00,22:00,2718,3.8,3.4,20,61.0,4.9,-3.0,1031.6,1034.2,2.1


# 2. 데이터 가공하기 

## 1) datetime 컬럼 제작, datetime 형식으로 변환

In [228]:
df = allData.copy()
df['datetime'] = pd.to_datetime(df['datetime'],format='%Y-%m-%d %H:%M', errors='ignore') #합친거 datetime 형식으로 변환
df['date'] = pd.to_datetime(df['date'],format='%Y-%m-%d', errors='ignore')  #date형식으로 변환
#df['time'] = pd.to_datetime(df['time'],format='%H:%M', errors='ignore') #time 형식으로 변환
df.head(5)

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem
0,2013-01-01 00:00:00,2013-01-01,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0
1,2013-01-01 01:00:00,2013-01-01,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0
2,2013-01-01 02:00:00,2013-01-01,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0
3,2013-01-01 03:00:00,2013-01-01,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3
4,2013-01-01 04:00:00,2013-01-01,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1


## 2) 년(year), 월(month), 일(day), 계절(season) 컬럼 생성



In [229]:
df['year'] = df['date'].dt.year # 년
df['month'] = df['date'].dt.month # 월
df['day'] = df['date'].dt.day# 일

conditionlist = [
    (df['month'] == 3)|(df['month'] == 4)| (df['month'] == 5) ,
    (df['month'] == 6)|(df['month'] == 7)|(df['month'] == 8) ,
    (df['month'] == 9)|(df['month'] == 10)|(df['month'] == 11),
    (df['month'] == 12)|(df['month'] == 1)|(df['month'] == 2) ]
choicelist = ['0', '1', '2','3']
#choicelist = ['spring', 'summer', 'autumn','winter']
df['season'] = np.select(conditionlist, choicelist, default='Not Specified')
df.head(5)

,datetime,date,time,windForce,tem,windSpeed,windDirect,humidity,vaporPressure,dewPoint,pressure,seaLevelPressure,groundTem,year,month,day,season
0,2013-01-01 00:00:00,2013-01-01,00:00,0,1.3,3.2,340,77.0,5.2,-2.2,1019.1,1021.3,0.0,2013,1,1,3
1,2013-01-01 01:00:00,2013-01-01,01:00,0,0.9,2.6,320,77.0,5.0,-2.6,1019.0,1021.2,0.0,2013,1,1,3
2,2013-01-01 02:00:00,2013-01-01,02:00,92,1.5,3.0,340,77.0,5.2,-2.1,1019.0,1021.2,0.0,2013,1,1,3
3,2013-01-01 03:00:00,2013-01-01,03:00,269,2.7,1.7,270,74.0,5.5,-1.4,1019.0,1021.2,0.3,2013,1,1,3
4,2013-01-01 04:00:00,2013-01-01,04:00,541,0.6,1.8,320,82.0,5.2,-2.1,1018.5,1020.8,0.1,2013,1,1,3


# 4. 훈련 & 예측

## 1) train-test data 나누기

In [230]:
df2 = df.copy()
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52584 entries, 0 to 52583
Data columns (total 17 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   datetime          52584 non-null  datetime64[ns]
 1   date              52584 non-null  datetime64[ns]
 2   time              52584 non-null  object        
 3   windForce         52584 non-null  int64         
 4   tem               52584 non-null  float64       
 5   windSpeed         52584 non-null  float64       
 6   windDirect        52584 non-null  int64         
 7   humidity          52584 non-null  float64       
 8   vaporPressure     52584 non-null  float64       
 9   dewPoint          52584 non-null  float64       
 10  pressure          52584 non-null  float64       
 11  seaLevelPressure  52584 non-null  float64       
 12  groundTem         52584 non-null  float64       
 13  year              52584 non-null  int64         
 14  month             5258

In [231]:
X = df2[['tem','windSpeed','windDirect','humidity']] #,'groundTem','seaLevelPressure'
y = df2[['windForce']]

transform = MinMaxScaler()
trans_X=transform.fit_transform(X)
trans_y=transform.fit_transform(y)

X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=10)
print('train data 개수 : ', len(X_train))
print('test data 개수: ', len(X_test))
print(len(X_train)+len(X_test))

train data 개수 :  42067
test data 개수:  10517
52584


In [232]:
X_train.ndim
X_train.shape

(42067, 4)

## 5. MakeDataSet


In [233]:
# https://teddylee777.github.io/tensorflow/LSTM%EC%9C%BC%EB%A1%9C-%EC%98%88%EC%B8%A1%ED%95%B4%EB%B3%B4%EB%8A%94-%EC%82%BC%EC%84%B1%EC%A0%84%EC%9E%90-%EC%A3%BC%EA%B0%80
def make_dataset(data, label, window_size=1):
    feature_list = []
    label_list = []
    for i in range(len(data) - window_size):
        feature_list.append(np.array(data.iloc[i:i+window_size]))
        label_list.append(np.array(label.iloc[i+window_size]))
    return np.array(feature_list), np.array(label_list)

In [234]:
# X = df2[['tem','windSpeed','humidity','groundTem','windDirect','seaLevelPressure']] #,'windDirect'
# X = df2[['tem','windSpeed','groundTem']] #,'windDirect'
transform = MinMaxScaler()

scale_cols=['tem','windSpeed','groundTem','windForce']
df_scaled = df2[['tem','windSpeed','groundTem','windForce']] 
df_scaled = transform.fit_transform(df_scaled)
df_scaled = pd.DataFrame(df_scaled)
df_scaled.columns = scale_cols

train = df_scaled[:int(len(df2)*0.8)]
test = df_scaled[int(len(df2)*0.8):]
print(train.head())
print(test.head())

        tem  windSpeed  groundTem  windForce
0  0.190123   0.180791   0.061135   0.000000
1  0.180247   0.146893   0.061135   0.000000
2  0.195062   0.169492   0.061135   0.007663
3  0.224691   0.096045   0.065502   0.022407
4  0.172840   0.101695   0.062591   0.045065
            tem  windSpeed  groundTem  windForce
42067  0.607407   0.158192   0.312955   0.144940
42068  0.585185   0.062147   0.305677   0.111370
42069  0.565432   0.067797   0.298399   0.079967
42070  0.558025   0.073446   0.292576   0.061808
42071  0.562963   0.079096   0.289665   0.055560


In [235]:
WINDOW_SIZE = 1 # 일 데이터를 묶을지 (window_size : 20, 1)
feature1 = ['tem','windSpeed','groundTem']
labels1 = ['windForce']

train_feature = train[feature1]
train_label = train[labels1]

train_feature, train_label = make_dataset(train_feature, train_label, WINDOW_SIZE)

X_train, X_valid,y_train, y_valid = train_test_split(train_feature,train_label,test_size=0.2,random_state=10) # train, valid
print(X_train.shape, X_valid.shape)

(33652, 1, 3) (8414, 1, 3)


In [236]:
test_feature = test[feature1]
test_label = test[labels1]
test_feature, test_label = make_dataset(test_feature, test_label, WINDOW_SIZE)
test_feature.shape, test_label.shape

((10516, 1, 3), (10516, 1))

### model loss function

In [237]:
from sklearn.metrics import mean_squared_error
def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))

def MSE(y_test, y_predict):
    return mean_squared_error(y_test, y_predict)

### nn model

In [238]:
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.layers import LSTM
import os

In [239]:
nn_model = Sequential()
nn_model.add(Dense(128, activation='relu'))
nn_model.add(Dense(1, activation='sigmoid'))

# network.add(layers.Dense(128, input_shape=train_feature.shape,activation='relu'))
# network.add(layers.Dense(64,activation='relu'))
# network.add(layers.Dense(1))

In [ ]:
nn_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
early_stop = EarlyStopping(monitor='val_loss', patience=5)
model_path = '/content/gdrive/My Drive/Colab Notebooks'
filename = os.path.join(model_path, 'tmp_nn_model1.h5')
print(filename)
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = nn_model.fit(X_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(X_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])


/content/gdrive/My Drive/Colab Notebooks/tmp_nn_model1.h5
Epoch 1/200
2104/2104 [==============================] - 4s 2ms/step - loss: 0.0592 - mae: 0.1966 - acc: 0.1193 - val_loss: 0.0422 - val_mae: 0.1508 - val_acc: 0.1171

Epoch 00001: val_loss improved from inf to 0.04220, saving model to /content/gdrive/My Drive/Colab Notebooks/tmp_nn_model1.h5
Epoch 2/200
2104/2104 [==============================] - 3s 2ms/step - loss: 0.0427 - mae: 0.1531 - acc: 0.1201 - val_loss: 0.0419 - val_mae: 0.1515 - val_acc: 0.1171

Epoch 00002: val_loss improved from 0.04220 to 0.04188, saving model to /content/gdrive/My Drive/Colab Notebooks/tmp_nn_model1.h5
Epoch 3/200
2104/2104 [==============================] - 3s 1ms/step - loss: 0.0425 - mae: 0.1541 - acc: 0.1196 - val_loss: 0.0419 - val_mae: 0.1487 - val_acc: 0.1171

Epoch 00003: val_loss did not improve from 0.04188
Epoch 4/200
2104/2104 [==============================] - 3s 2ms/step - loss: 0.0417 - mae: 0.1518 - acc: 0.1197 - val_loss: 0.0418 

In [ ]:
# weight 로딩
nn_model.load_weights(filename)

# 예측
nn_pred = nn_model.predict(test_feature)

In [ ]:
plt.figure(figsize=(16,10))
ax1 = sns.distplot(test_label,hist=False,label='y')
ax2 = sns.distplot(nn_pred,hist=False,label='y_hat',ax=ax1)

plt.show()
plt.close()

In [ ]:
print(MSE(test_label, pred))
#print(RMSE(test_label, pred))
loss, acc, dfd = nn_model.evaluate(X_train, y_train)
loss

### lstm model

In [ ]:
lstm_model = Sequential()
lstm_model.add(LSTM(128,input_shape=(train_feature.shape[1], train_feature.shape[2]), activation='relu', return_sequences=False))
lstm_model.add(Dense(1, activation='sigmoid'))

# network.add(layers.Dense(128, input_shape=train_feature.shape,activation='relu'))
# network.add(layers.Dense(64,activation='relu'))
# network.add(layers.Dense(1))

In [ ]:
lstm_model.compile(loss='mean_squared_error', optimizer='adam',metrics=['mae', 'acc'])
early_stop = EarlyStopping(monitor='val_loss', patience=5)
model_path = '/content/gdrive/My Drive/Colab Notebooks'
filename = os.path.join(model_path, 'tmp_lstm1.h5')
print(filename)
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='auto')

history = lstm_model.fit(X_train, y_train, 
                    epochs=200, 
                    batch_size=16,
                    validation_data=(X_valid, y_valid), 
                    callbacks=[early_stop, checkpoint])


In [ ]:
# weight 로딩
lstm_model.load_weights(filename)

# 예측
pred = lstm_model.predict(test_feature)

In [ ]:
plt.figure(figsize=(12, 9))
plt.plot(test_label, label='actual')
plt.plot(pred, label='prediction')
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(16,10))
ax1 = sns.distplot(test_label,hist=False,label='y')
ax2 = sns.distplot(pred,hist=False,label='y_hat',ax=ax1)

plt.show()
plt.close()

In [ ]:
print(MSE(test_label, pred)) 
#print(RMSE(test_label, pred))
loss, acc, dfd = lstm_model.evaluate(X_train, y_train)
loss

In [ ]:
, stateful=True)